<a href="https://colab.research.google.com/github/EEG-Easy/EEG-Machinelearning/blob/master/2020-2Project/N100%2CP200Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import tensorflow as tf
tf.enable_eager_execution()
from tensorflow import keras
import os

import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

tf.keras.backend.clear_session()

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
cd /content/gdrive/My Drive/eeg

/content/gdrive/My Drive/eeg


In [0]:
EEG_df = pd.read_csv('mergedTrialData.csv')

In [5]:
EEG_df = EEG_df.drop(['subject', 'trial','Fz_P200', 'FCz_P200', 'Cz_P200',
       'FC3_P200', 'FC4_P200', 'C3_P200', 'C4_P200', 'CP3_P200', 'CP4_P200',
       'AFz_P200', 'F2_P200', 'F1_P200', 'FC1_P200'], axis=1)
EEG_df.columns

Index(['condition', 'group', 'Fz_N100', 'FCz_N100', 'Cz_N100', 'FC3_N100',
       'FC4_N100', 'C3_N100', 'C4_N100', 'CP3_N100', 'CP4_N100', 'AFz_N100',
       'F2_N100', 'F1_N100', 'FC1_N100'],
      dtype='object')

In [0]:
train, test = train_test_split(EEG_df, test_size=0.1, random_state=123)

In [0]:
condition_train=np.array(train['condition'])
group_train=np.array(train['group'])
#trial_train=np.array(train['trial'])

train_scaling = train.drop(['condition','group'], axis=1)

In [0]:
condition_test=np.array(test['condition'])
group_test=np.array(test['group'])
#trial_test=np.array(test['trial'])

test_scaling = test.drop(['condition','group'], axis=1)

In [0]:
scaler = RobustScaler()

In [0]:
scaler.fit(train_scaling) 

train_scaled = scaler.transform(train_scaling)
test_scaled = scaler.transform(test_scaling)

train_scaled_df = pd.DataFrame(train_scaled)
test_scaled_df = pd.DataFrame(test_scaled)

In [11]:
train_scaled_df.columns = ['Fz_N100', 'FCz_N100', 'Cz_N100', 'FC3_N100', 'FC4_N100', 'C3_N100', 'C4_N100', 'CP3_N100', 'CP4_N100', 'AFz_N100', 'F2_N100', 'F1_N100', 'FC1_N100']
condition_train = condition_train.astype(int)
group_train = group_train.astype(int)
#trial_train = trial_train.astype(int)

train_scaled_df['condition'] = condition_train
train_scaled_df['group'] = group_train
#train_scaled_df['trial'] = trial_train
train_scaled_df.head(5)

,Fz_N100,FCz_N100,Cz_N100,FC3_N100,FC4_N100,C3_N100,C4_N100,CP3_N100,CP4_N100,AFz_N100,F2_N100,F1_N100,FC1_N100,condition,group
0,-1.908729,-1.943668,-0.929460,-1.271370,-1.743739,-0.611202,0.391713,0.258216,0.613002,-2.064743,-2.134181,-1.707939,-1.716523,2,1
1,0.347628,0.521196,0.259843,0.369615,-0.158861,0.257364,-0.280448,0.080719,-0.236345,1.588914,-0.210472,0.007524,0.386791,3,0
2,0.184458,0.114874,0.290263,0.169201,0.255214,0.337890,0.208438,0.286272,0.201832,0.349909,0.371388,0.243771,0.395103,3,1
3,-0.400977,-0.412909,-0.469208,-0.105697,-1.018351,-0.569663,-0.426318,-0.479960,-0.580529,0.198097,-0.322636,-0.547395,-0.522286,1,0
4,0.565459,0.363935,0.242151,0.390421,0.341408,0.101748,0.243033,-0.868456,0.230275,0.444669,0.420095,0.788671,-0.360881,1,0


In [12]:
test_scaled_df.columns = ['Fz_N100', 'FCz_N100', 'Cz_N100', 'FC3_N100', 'FC4_N100', 'C3_N100', 'C4_N100', 'CP3_N100', 'CP4_N100', 'AFz_N100', 'F2_N100', 'F1_N100', 'FC1_N100']
condition_test = condition_test.astype(int)
group_test = group_test.astype(int)
#trial_test = trial_test.astype(int)

test_scaled_df['condition'] = condition_test
test_scaled_df['group'] = group_test
#test_scaled_df['trial'] = trial_test
test_scaled_df.head(5)

,Fz_N100,FCz_N100,Cz_N100,FC3_N100,FC4_N100,C3_N100,C4_N100,CP3_N100,CP4_N100,AFz_N100,F2_N100,F1_N100,FC1_N100,condition,group
0,-1.113170,-0.445164,-0.546663,-0.296657,-0.189324,-0.202174,-0.355104,-0.991689,-0.675795,-0.889946,-1.307404,-0.743348,-0.213663,1,0
1,-0.390938,-0.433051,-0.640184,-0.297166,-0.554171,-0.336713,-0.900643,-0.414222,-0.930191,-0.407668,-0.356771,-0.222774,-0.269992,1,0
2,-0.314446,-0.093463,-0.329860,-0.901320,0.421864,-0.491504,0.000279,-0.408203,-0.313209,-0.315266,-0.153139,-0.941053,-0.047754,3,0
3,-0.475385,-0.686775,-1.419755,-1.279144,-0.620453,-1.277601,-1.371880,-1.736170,-1.970691,0.033358,-0.264970,-0.461537,-0.950308,2,1
4,-0.245906,-0.626354,-0.754979,-0.472125,-0.216677,-0.420495,-0.154642,-0.286900,-0.754872,-0.046324,-0.169822,-0.156313,-0.537985,3,1


In [0]:
train_scaled_df.to_csv('/content/gdrive/My Drive/eeg/train_mergedTrial.csv',sep=',', na_rep='NaN')
test_scaled_df.to_csv('/content/gdrive/My Drive/eeg/test_mergedTrial.csv',sep=',', na_rep='NaN')

In [14]:
LABEL_COLUMN = 'group'
LABELS = [0, 1]

CSV_COLUMNS = ['condition', 'group', 'Fz_N100', 'FCz_N100', 'Cz_N100', 'FC3_N100', 'FC4_N100', 'C3_N100', 'C4_N100', 'CP3_N100', 'CP4_N100', 'AFz_N100', 'F2_N100', 'F1_N100', 'FC1_N100']

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=32, 
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

train_dataset = get_dataset('train_mergedTrial.csv', select_columns=CSV_COLUMNS)
test_dataset = get_dataset('test_mergedTrial.csv', select_columns=CSV_COLUMNS)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['Fz_N100', 'FCz_N100', 'Cz_N100', 'FC3_N100', 'FC4_N100', 'C3_N100', 'C4_N100', 'CP3_N100', 'CP4_N100', 'AFz_N100', 'F2_N100', 'F1_N100', 'FC1_N100']

train_data = train_dataset.map(
    PackNumericFeatures(NUMERIC_FEATURES))
test_data = test_dataset.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [17]:
import pandas as pd
desc = pd.read_csv('train_mergedTrial.csv')[NUMERIC_FEATURES].describe()
desc

,Fz_N100,FCz_N100,Cz_N100,FC3_N100,FC4_N100,C3_N100,C4_N100,CP3_N100,CP4_N100,AFz_N100,F2_N100,F1_N100,FC1_N100
count,2.088000e+04,2.088000e+04,20880.000000,2.088000e+04,2.088000e+04,20880.000000,2.088000e+04,20880.000000,20880.000000,2.088000e+04,2.088000e+04,20880.000000,2.088000e+04
mean,-7.822770e-04,-2.478773e-03,-0.010972,7.730227e-04,7.082304e-03,-0.004372,4.661683e-03,-0.002541,0.003509,1.473456e-02,1.408067e-03,-0.002048,-2.213974e-03
std,9.303461e-01,8.737721e-01,0.861491,9.002744e-01,9.032784e-01,0.879932,8.774937e-01,0.878770,0.877921,1.054618e+00,9.373293e-01,0.960848,9.009437e-01
min,-1.595465e+01,-1.360312e+01,-10.710464,-1.432177e+01,-1.388155e+01,-12.615774,-1.324899e+01,-10.365569,-10.351497,-1.995685e+01,-1.661223e+01,-18.418743,-2.023526e+01
25%,-5.098435e-01,-5.092074e-01,-0.509078,-4.957379e-01,-4.973867e-01,-0.506255,-5.007309e-01,-0.498152,-0.498496,-5.053183e-01,-5.065945e-01,-0.507522,-5.029683e-01
50%,-1.813752e-17,-1.707618e-17,0.000000,-9.798477e-18,1.937418e-17,0.000000,-9.819229e-18,0.000000,0.000000,-2.020004e-17,1.868385e-17,0.000000,1.799098e-17
75%,4.901565e-01,4.907926e-01,0.490922,5.042621e-01,5.026133e-01,0.493745,4.992691e-01,0.501848,0.501504,4.946817e-01,4.934055e-01,0.492478,4.970317e-01
max,1.831839e+01,1.060453e+01,8.164751,1.373160e+01,1.778229e+01,10.574900,1.121623e+01,9.410619,9.035364,2.519133e+01,1.633935e+01,16.104661,1.141266e+01


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [20]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(13,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f648ba83e18>, mean=array([-0.00078228, -0.00247877, -0.01097214,  0.00077302,  0.0070823 ,
       -0.00437241,  0.00466168, -0.00254084,  0.00350858,  0.01473456,
        0.00140807, -0.00204818, -0.00221397]), std=array([0.9303461 , 0.87377213, 0.86149076, 0.90027443, 0.90327843,
       0.87993166, 0.87749373, 0.87877018, 0.87792124, 1.05461781,
       0.93732932, 0.96084788, 0.90094367])))

In [21]:
example_batch, labels_batch = next(iter(train_data)) 
example_batch['numeric']

<tf.Tensor: id=248, shape=(32, 13), dtype=float32, numpy=
array([[-2.28631210e+00, -1.75679481e+00, -1.30675030e+00,
        -1.62165844e+00, -1.33035731e+00, -9.46616471e-01,
        -7.95863092e-01, -4.86307859e-01, -3.47871870e-01,
        -2.25631094e+00, -1.93460798e+00, -2.15763330e+00,
        -1.98787582e+00],
       [ 5.14298797e-01,  3.78493965e-01,  6.11680210e-01,
        -1.57147095e-01,  2.52337009e-01,  6.29275560e-01,
         3.31153274e-01,  1.92887798e-01, -1.05674826e-01,
         2.22424462e-01,  2.48710379e-01,  2.41548762e-01,
         1.28705785e-01],
       [-8.34523499e-01, -9.81073380e-01, -9.89702523e-01,
        -6.79381013e-01, -7.61388898e-01, -6.38895988e-01,
        -7.95370698e-01, -3.14146489e-01, -9.56233859e-01,
        -3.31531703e-01, -7.62208939e-01, -5.85048497e-01,
        -8.95972669e-01],
       [ 8.91623318e-01,  8.65138233e-01,  8.56508076e-01,
         1.09665203e+00,  6.71957135e-01,  1.15307522e+00,
         7.29044497e-01,  1.26772821e+

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)

In [0]:
CATEGORIES = {'condition': [1, 2, 3]}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns) #+numeric_columns

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='softmax'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [28]:
checkpoint_path = "model4/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,

    period=20)

In [31]:
model.fit(train_data, epochs=20, callbacks = [cp_callback])

Epoch 1/20
653/653 [==============================] - 6s 8ms/step - loss: 6.0220 - acc: 0.6051
Epoch 2/20
653/653 [==============================] - 3s 4ms/step - loss: 6.0225 - acc: 0.6051
Epoch 3/20
653/653 [==============================] - 3s 4ms/step - loss: 6.0225 - acc: 0.6051
Epoch 4/20
653/653 [==============================] - 3s 4ms/step - loss: 6.0225 - acc: 0.6051
Epoch 5/20
453/653 [===================>..........] - ETA: 0s - loss: 6.0646 - acc: 0.6023

KeyboardInterrupt: ignored

In [0]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

In [0]:
model.add(tf.keras.layers.Dropout(0.3))